In [1]:
import torch

from model import Knn_Kmeans_Logits 
from utils.other import load_data
from configs.resnet_config import config

In [2]:
device =  torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model = Knn_Kmeans_Logits(config, device=device)

for i in range(6):

    X_train, y_train, X_test, y_test, covariances = load_data(task_num=i, dataset_name="resnet", load_covariances=True, load_prototypes=False)

    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    acc = (torch.sum((y_test.flatten().to(device)==predictions).int()) / X_test.shape[0] * 100).item()

    print(f"Accuracy: {acc} MY\n")


Accuracy: 68.48, Task ID: 0
Accuracy: 68.41999816894531 MY

Accuracy: 62.55, Task ID: 1
Accuracy: 62.54999542236328 MY

Accuracy: 60.7, Task ID: 2
Accuracy: 60.70000076293945 MY

Accuracy: 58.1, Task ID: 3
Accuracy: 58.10000228881836 MY

Accuracy: 55.13333333333333, Task ID: 4
Accuracy: 55.13333511352539 MY

Accuracy: 52.49, Task ID: 5
Accuracy: 52.48999786376953 MY

